# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

In [1]:
# ETL Cities
import pandas as pd
import numpy as np
import re

def etl_cities(team,loc):
    team = team.upper()
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]":"Population"},inplace=True)

    # delete "[...]"
    cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]","", regex=True)
    cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]","", regex=True)
    cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]","", regex=True)
    cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]","", regex=True)

    cities_aux = cities[['Metropolitan area',team]]

    teams = cities_aux[team].unique()
    indexes = []
    for i in teams:
        m = re.findall(r"^[A-Za-z0-9]",i)
        if len(m)>0:
            indexes.append(cities_aux[cities_aux[team]==i].index.values[0])

    cities_aux = cities_aux.iloc[indexes]
    cities_w = cities_aux.iloc[0:loc]
    met_area = cities_w['Metropolitan area'].values
    team_values = cities_w[team].values
    cities_aux = cities_aux.drop(np.arange(0,loc,1))

    ind = 0
    for i in range(0,len(met_area)):
        m = re.findall("[A-Z0-9]*[a-z]+",team_values[i])
        for n in m:
            df = pd.DataFrame({'Metropolitan area' : met_area[i] , team : n},index=[ind])
            cities_aux=pd.concat([cities_aux,df])
            ind+=1

    cities_aux = cities_aux.reset_index().drop('index', axis=1)

    population = cities[['Metropolitan area','Population']]

    cities_aux = cities_aux.merge(population, how='left', on='Metropolitan area')
    cities_aux=cities_aux.rename(columns={team:'team'})

    cities_aux['team']=cities_aux['team'].str.replace('[\w]*\ ','', regex=True)

    return cities_aux

In [2]:
def etl_big4(team):

    path = "assets/{}.csv".format(team.lower())
    big4_df =pd.read_csv(path)

    big4_df = big4_df[big4_df.year ==2018]
    big4_df = big4_df[['team','W','L']]

    indexes = []
    for i in range(big4_df.shape[0]):
        row=big4_df.iloc[i]
        if row['team']==row['W'] and row['L']==row['W']:
            indexes.append(i)
    if len(indexes)>0:
        big4_df = big4_df.drop(indexes)

    big4_df['team'] = big4_df['team'].str.replace('\*','', regex=True)
    big4_df['team'] = big4_df['team'].str.replace('\s\(\d*\)','', regex=True)
    big4_df['team'] = big4_df['team'].str.replace('[\w.]*\ ','',regex=True)

    
    big4_df = big4_df.astype({'team': str,'W': int, 'L': int})
    
    big4_df['WLRatio'] = big4_df['W']/(big4_df['W']+big4_df['L'])

    return big4_df

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [21]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")


def nhl_correlation(): 
    # YOUR CODE HERE
    cities_nhl = etl_cities('NHL',2)

    nhl_2018 = etl_big4('nhl')
    
    q1_merge=pd.merge(cities_nhl,nhl_2018,'outer', on = 'team')
    q1_merge = q1_merge.groupby('Metropolitan area').agg({'WLRatio': np.nanmean, 'Population': np.nanmean})

    # raise NotImplementedError()
    
    population_by_region = q1_merge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = q1_merge['WLRatio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)
    raise NotImplementedError()
nhl_correlation()

(-0.08537702363136182, 0.6657663663986723)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [25]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    cities_nba = etl_cities('NBA',2)
    nba_2018 = etl_big4('nba')
    q2_merge=pd.merge(cities_nba,nba_2018,'outer', on = 'team')
    q2_merge = q2_merge.groupby('Metropolitan area').agg({'WLRatio': np.nanmean, 'Population': np.nanmean})


    population_by_region = q2_merge['Population']  # pass in metropolitan area population from cities
    win_loss_by_region = q2_merge['WLRatio']  # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)
    raise NotImplementedError()

In [26]:
nba_correlation()

(-0.2021940882816165, 0.302148236118873)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [27]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    mlb_2018 = etl_big4('mlb')
    mlb_2018.at[0,'team']='Red Sox'
    mlb_2018.at[8,'team']='White Sox'

    cities_mlb = etl_cities('MLB',4)
    cities_mlb.at[30,'team']='White Sox'
    cities_mlb.at[3,'team']='Red Sox'

    q3_merge=pd.merge(cities_mlb,mlb_2018,'outer', on = 'team')
    q3_merge = q3_merge.groupby('Metropolitan area').agg({'WLRatio': np.nanmean, 'Population': np.nanmean})

     
    population_by_region = q3_merge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = q3_merge['WLRatio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)
    
    raise NotImplementedError()
   

In [28]:
mlb_correlation()

(-0.032725738592627396, 0.8739016967469131)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [29]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    nfl_2018 = etl_big4('nfl')
    nfl_2018['team'] = nfl_2018['team'].str.replace('\+','', regex=True)

    cities_nfl = etl_cities('nfl',3)
    
    q4_merge=pd.merge(cities_nfl,nfl_2018,'outer', on = 'team')
    q4_merge = q4_merge.groupby('Metropolitan area').agg({'WLRatio': np.nanmean, 'Population': np.nanmean})
    

    population_by_region = q4_merge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = q4_merge['WLRatio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)
    raise NotImplementedError()

In [30]:
nfl_correlation()

(-0.049839450556883366, 0.7973739749607349)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from scipy.stats import ttest_rel

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

    #nhl
    cities_nhl = etl_cities('NHL',2)
    nhl_2018 = etl_big4('nhl')
    q1_merge=pd.merge(cities_nhl,nhl_2018,'outer', on = 'team')
    q1_merge = q1_merge.groupby('Metropolitan area').agg({'WLRatio': np.nanmean, 'Population': np.nanmean})

    #nba
    cities_nba = etl_cities('NBA',2)
    nba_2018 = etl_big4('nba')
    q2_merge=pd.merge(cities_nba,nba_2018,'outer', on = 'team')
    q2_merge = q2_merge.groupby('Metropolitan area').agg({'WLRatio': np.nanmean, 'Population': np.nanmean})

    #mlb
    mlb_2018 = etl_big4('mlb')
    mlb_2018.at[0,'team']='Red Sox'
    mlb_2018.at[8,'team']='White Sox'

    cities_mlb = etl_cities('MLB',4)
    cities_mlb.at[30,'team']='White Sox'
    cities_mlb.at[3,'team']='Red Sox'

    q3_merge=pd.merge(cities_mlb,mlb_2018,'outer', on = 'team')
    q3_merge = q3_merge.groupby('Metropolitan area').agg({'WLRatio': np.nanmean, 'Population': np.nanmean})

    #nfl
    nfl_2018 = etl_big4('nfl')
    nfl_2018['team'] = nfl_2018['team'].str.replace('\+','', regex=True)

    cities_nfl = etl_cities('nfl',3)

    q4_merge=pd.merge(cities_nfl,nfl_2018,'outer', on = 'team')
    q4_merge = q4_merge.groupby('Metropolitan area').agg({'WLRatio': np.nanmean, 'Population': np.nanmean})

    # all merges

    nhl_merge = q1_merge.drop('Population',axis=1)
    nba_merge = q2_merge.drop('Population',axis=1)
    mlb_merge = q3_merge.drop('Population',axis=1)
    nfl_merge = q4_merge.drop('Population',axis=1)

    def to_merge(sport):
        if sport == 'NFL':
            return nfl_merge
        elif sport == 'NBA':
            return nba_merge
        elif sport == 'NHL':
            return nhl_merge
        elif sport == 'MLB':
            return mlb_merge
   
    for i in sports:
        for j in sports:
            merged=pd.merge(to_merge(i),to_merge(j),'inner', on = ['Metropolitan area'])
            p_values.loc[i,j]=ttest_rel(merged['WLRatio_x'], merged['WLRatio_y'])[1]

    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
    raise NotImplementedError()


In [6]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
